In [ ]:
import os
import sys
import torch
import torchaudio
import numpy as np
from src.CQT_nsgt import CQT_nsgt
import soundfile as sf
import math
import time
torch.set_printoptions(linewidth=2000, threshold=170000)
np.set_printoptions(threshold=sys.maxsize)

: 

In [8]:
#STEREO example
example="/scratch/work/molinee2/projects/ddpm/diffusion_autumn_2022/CQTdiff/test_dir/44100_long/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.wav"
#example="/scratch/work/molinee2/projects/ddpm/diffusion_autumn_2022/CQTdiff/test_dir/22050_long/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.wav"
a,fs=sf.read(example)
#a=0.5*(a[...,0]+a[...,1])
#resample=torch.resample()
a=a.transpose()
x=torch.Tensor(a)
x=torchaudio.functional.resample(x, 44100, 22050)

x=x.unsqueeze(0).repeat(1,1,1)
print(x.shape)

numocts=8
binsoct=65

Ls=65536 # most efficient one
cqt=CQT_nsgt(numocts, binsoct, mode="oct",fs=fs, audio_len=Ls)

torch.Size([1, 2, 3114069])
1024 697


In [2]:
#MONO example
#example="/scratch/work/molinee2/projects/ddpm/diffusion_autumn_2022/CQTdiff/test_dir/44100_long/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.wav"
example="/scratch/work/molinee2/projects/ddpm/diffusion_autumn_2022/CQTdiff/test_dir/22050_long/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_04_WAV.wav"
a,fs=sf.read(example)
#a=0.5*(a[...,0]+a[...,1])
#a=a.transpose()
#x=torch.Tensor(a)
#x=torchaudio.functional.resample(x, 44100, 22050)
#x.shape
x=torch.Tensor(a).unsqueeze(0) #unsqueeze stereo dimension
x=x.unsqueeze(0) #unsqueeze batch dim

numocts=8
binsoct=32

Ls=88200 # most efficient one
cqt=CQT_nsgt(numocts, binsoct, mode="oct",fs=fs, audio_len=Ls)

2048 1877


In [3]:
x=x[...,44100:(44100+Ls)]

In [4]:
cqt

In [5]:
#x=x[...,44100:(44100+Ls)]
start=time.time()
#X=forward(x)
X=cqt.fwd(x)
end=time.time()
print(end-start)

0.14071321487426758


In [6]:
for a in X:
    print(a.shape)

torch.Size([1, 1, 1, 345])
torch.Size([1, 1, 1, 176])
torch.Size([1, 1, 32, 16])
torch.Size([1, 1, 32, 32])
torch.Size([1, 1, 32, 64])
torch.Size([1, 1, 32, 128])
torch.Size([1, 1, 32, 256])
torch.Size([1, 1, 32, 512])
torch.Size([1, 1, 32, 1024])
torch.Size([1, 1, 32, 2048])


In [7]:
start=time.time()
#xrec=backward(X)
#xrec=nsigtf(X, cqt.gd, cqt.wins, cqt.nn, Ls=Ls, mode=cqt.mode, device=cqt.device)
xrec=cqt.bwd(X)
end=time.time()
print(end-start)

0.043013811111450195


In [8]:
print(x.shape, xrec.shape)

torch.Size([1, 1, 88200]) torch.Size([1, 1, 88200])


In [ ]:
A=torch.stft(x[0,0], 1024)
Arec=torch.stft(xrec[0,0], 1024)
error=A[:,:,:]-Arec[:,:,:]
Error=torch.sqrt(error[...,1]**2+error[...,0]**2).squeeze(0)
import plotly.express as px
px.imshow(10*torch.log10(Error))


/share/apps/anaconda-ci/fgci-centos7-anaconda/software/anaconda/2022-03/388ac2b9/lib/python3.9/site-packages/torch/functional.py:606: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484806139/work/aten/src/ATen/native/SpectralOps.cpp:800.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [71]:
A=torch.stft(x[1,1], 1024)
Arec=torch.stft(xrec[1,1], 1024)
error=A[:,:,:]-Arec[:,:,:]
Error=torch.sqrt(error[...,1]**2+error[...,0]**2).squeeze(0)
import plotly.express as px
px.imshow(10*torch.log10(Error))

IndexError: index 1 is out of bounds for dimension 0 with size 1

NameError: name 'ipd' is not defined

In [14]:
import IPython.display as ipd
ipd.Audio(xrec[0], rate=22050, autoplay=True) # load a NumPy array

In [18]:
ipd.Audio(x[0], rate=22050, autoplay=True) # load a NumPy array

In [17]:
x.shape

torch.Size([1, 2, 65536])

In [9]:
450//2 +1

226